## Initialising SageMaker Resources

In [1]:
import sagemaker, boto3, json
# from sagemaker import get_execution_role

aws_role = "arn:aws:iam::654654375075:role/service-role/AmazonSageMaker-ExecutionRole-20240502T152077"
aws_region = boto3.Session().region_name
sess = sagemaker.Session()
bucket = sess.default_bucket()

sagemaker.config INFO - Not applying SDK defaults from location: /app/etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sprince0031/.var/app/com.visualstudio.code/config/sagemaker/config.yaml


In [ ]:
print(bucket)

sagemaker-us-east-1-654654375075


## Choosing a pretrained SageMaker Jumpstart model
A list of available Jumpstart models can be found from the models_manifest.json file that was downloaded using the [sample code from section 2 of their github example notebook](https://github.com/aws/amazon-sagemaker-examples/blob/main/introduction_to_amazon_algorithms/jumpstart_image_classification/Amazon_JumpStart_Image_Classification.ipynb). The list is also available in the [SageMaker Jumpstart documentation page](https://sagemaker.readthedocs.io/en/v2.82.0/doc_utils/jumpstart.html). We have chosen the latest available version of the Pytorch Resnet18 image classifier.

In [2]:
(
    model_id,
    model_version,
) = (
    "pytorch-ic-resnet18",
    "*",
)


## Deploying the Jumpstart model of choice

In [4]:
from sagemaker.jumpstart.model import JumpStartModel

my_model = JumpStartModel(model_id=model_id, role=aws_role, sagemaker_session=sess, region=aws_region)
base_model_predictor = my_model.deploy()

Using model 'pytorch-ic-resnet18' with wildcard version identifier '*'. You can pin to version '3.0.0' for more stable results. Note that models may have different input/output signatures after a major version upgrade.


-------!

In [6]:
base_model_predictor.accept = "application/json;verbose"

## Testing the deployed model on the SageMaker endpoint

In [7]:
s3_bucket = f"jumpstart-cache-prod-{aws_region}"
key_prefix = "inference-notebook-assets"


def download_from_s3(images):
    for filename, image_key in images.items():
        boto3.client("s3").download_file(s3_bucket, f"{key_prefix}/{image_key}", filename)


images = {"img1.jpg": "cat.jpg", "img2.jpg": "dog.jpg"}
download_from_s3(images)

In [8]:
from IPython.core.display import HTML


def predict_top_k_labels(probabilities, labels, k):
    topk_prediction_ids = sorted(
        range(len(probabilities)), key=lambda index: probabilities[index], reverse=True
    )[:k]
    topk_class_labels = ", ".join([labels[id] for id in topk_prediction_ids])
    return topk_class_labels


for image_filename in images.keys():
    with open(image_filename, "rb") as file:
        img = file.read()
    query_response = base_model_predictor.predict(img)
    labels, probabilities = query_response["labels"], query_response["probabilities"]
    top5_class_labels = predict_top_k_labels(probabilities, labels, 5)
    display(
        HTML(
            f'<img src={image_filename} alt={image_filename} align="left" style="width: 250px;"/>'
            f"<figcaption>Top-5 predictions: {top5_class_labels} </figcaption>"
        )
    )

## Deleting deployed SageMaker Endpoint resources
We don't want to have any costs associated for a PoC project. :) (Too late though, I already incured $0.68 :( )

In [5]:
# Delete the SageMaker endpoint and the attached resources
base_model_predictor.delete_model()
base_model_predictor.delete_endpoint()

---
## Lambda function: sagemakerEndpointInvoker

In [ ]:
import boto3
import json

def predict_top_k_labels(probabilities, labels, k):
    topk_prediction_ids = sorted(
        range(len(probabilities)), key=lambda index: probabilities[index], reverse=True
    )[:k]
    topk_class_labels = ", ".join([labels[id] for id in topk_prediction_ids])
    return topk_class_labels

def lambda_handler(event, context):
    try:
        # Initialize SageMaker runtime client
        sagemaker_runtime = boto3.client('sagemaker', region_name='us-east-1')

        # Get the image data from the event
        image_data = event['body']

        # Specify the SageMaker endpoint name
        ENDPOINT_NAME = os.environ['ENDPOINT_NAME'] # endpoint name saved under Configurations > Environment variables

        # Invoke the SageMaker endpoint
        response = sagemaker_runtime.invoke_endpoint(
            EndpointName=ENDPOINT_NAME,
            ContentType='image/jpeg',
            Body=image_data,
            
        )

        # Parse the inference result
        inference_result = json.loads(response['Body'].read().decode('utf-8'))
        
        labels, probabilities = inference_result["labels"], inference_result["probabilities"]
        top5_class_labels = predict_top_k_labels(probabilities, labels, 5)

        return {
            'statusCode': 200,
            'headers': {
                'Access-Control-Allow-Origin': '*'
            },
            'body': str(top5_class_labels)
        }
    except Exception as e:
        return {
            'statusCode': 500,
            'headers': {
                'Access-Control-Allow-Origin': '*'
            },
            'body': str(e)
        }


In [ ]:
from sagemaker.estimator import Estimator
from sagemaker.utils import name_from_base
from sagemaker.tuner import HyperparameterTuner

training_job_name = name_from_base(f"jumpstart-example-{model_id}-transfer-learning")

# Create SageMaker Estimator instance
ic_estimator = Estimator(
    role=aws_role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=train_model_uri,
    entry_point="transfer_learning.py",
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyperparameters,
    output_path=s3_output_location,
    base_job_name=training_job_name,
)

# Launch a SageMaker Training job by passing s3 path of the training data
ic_estimator.fit({"training": training_dataset_s3_path}, logs=True)